In [7]:

import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from PIL import Image
import zipfile
import glob

In [8]:
# Konfigurasi parameter
IMG_SIZE = 224
BATCH_SIZE = 128
EPOCHS = 5
LEARNING_RATE = 0.001
NUM_CLASSES = 2  # face dan non-face

In [9]:

class FaceDetectionModel:
    def __init__(self, img_size=224, num_classes=2):
        self.img_size = img_size
        self.num_classes = num_classes
        self.model = None
        self.history = None

    def create_cnn_model(self):
        """Membuat arsitektur CNN untuk deteksi wajah"""
        model = models.Sequential([
            # Convolutional layers
            layers.Conv2D(32, (3, 3), activation='relu', input_shape=(self.img_size, self.img_size, 3)),
            layers.BatchNormalization(),
            layers.MaxPooling2D((2, 2)),

            layers.Conv2D(64, (3, 3), activation='relu'),
            layers.BatchNormalization(),
            layers.MaxPooling2D((2, 2)),

            layers.Conv2D(128, (3, 3), activation='relu'),
            layers.BatchNormalization(),
            layers.MaxPooling2D((2, 2)),

            layers.Conv2D(256, (3, 3), activation='relu'),
            layers.BatchNormalization(),
            layers.MaxPooling2D((2, 2)),

            layers.Conv2D(512, (3, 3), activation='relu'),
            layers.BatchNormalization(),
            layers.MaxPooling2D((2, 2)),

            # Flatten and Dense layers
            layers.Flatten(),
            layers.Dropout(0.5),
            layers.Dense(512, activation='relu'),
            layers.Dropout(0.3),
            layers.Dense(256, activation='relu'),
            layers.Dropout(0.3),
            layers.Dense(self.num_classes, activation='softmax')
        ])

        self.model = model
        return model

    def compile_model(self, learning_rate=0.001):
        """Compile model dengan optimizer dan loss function"""
        optimizer = optimizers.Adam(learning_rate=learning_rate)
        self.model.compile(
            optimizer=optimizer,
            loss='categorical_crossentropy',
            metrics=['accuracy', 'precision', 'recall']
        )

    def create_data_generators(self, train_dir, validation_split=0.2):
        """Membuat data generator untuk training dan validation"""
        # Data augmentation untuk training
        train_datagen = ImageDataGenerator(
            rescale=1./255,
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            horizontal_flip=True,
            zoom_range=0.2,
            shear_range=0.2,
            brightness_range=[0.8, 1.2],
            validation_split=validation_split
        )

        # Generator untuk validation (hanya rescaling)
        val_datagen = ImageDataGenerator(
            rescale=1./255,
            validation_split=validation_split
        )

        train_generator = train_datagen.flow_from_directory(
            train_dir,
            target_size=(self.img_size, self.img_size),
            batch_size=BATCH_SIZE,
            class_mode='categorical',
            subset='training'
        )

        validation_generator = val_datagen.flow_from_directory(
            train_dir,
            target_size=(self.img_size, self.img_size),
            batch_size=BATCH_SIZE,
            class_mode='categorical',
            subset='validation'
        )

        return train_generator, validation_generator

def prepare_lfw_dataset():
    """Mempersiapkan dataset LFW untuk face detection"""
    base_path = '/home/wahyusetiawan/Documents/ProjectBelajar/DataAnalyst/facerecognition/lfw_funneled'
    faces_dir = os.path.join(base_path, 'faces')
    non_faces_dir = os.path.join(base_path, 'non_faces')
    final_dataset_dir = os.path.join(base_path, 'face_dataset')

    # Buat semua direktori yang dibutuhkan
    os.makedirs(faces_dir, exist_ok=True)
    os.makedirs(non_faces_dir, exist_ok=True)
    os.makedirs(final_dataset_dir, exist_ok=True)

    def generate_negative_samples():
        """Generate negative samples dari background crops"""
        face_images = []
        for person_dir in os.listdir(base_path):
            person_path = os.path.join(base_path, person_dir)
            if os.path.isdir(person_path) and person_dir not in ['faces', 'non_faces', 'face_dataset']:
                for img_file in os.listdir(person_path):
                    if img_file.lower().endswith(('.jpg', '.png')):
                        img_path = os.path.join(person_path, img_file)
                        face_images.append(img_path)

        negative_count = 0
        for img_path in face_images[:1000]:
            try:
                img = cv2.imread(img_path)
                if img is not None:
                    h, w = img.shape[:2]
                    for _ in range(2):
                        x = np.random.randint(0, max(1, w-50))
                        y = np.random.randint(0, max(1, h-50))
                        crop_w = np.random.randint(30, min(100, w-x))
                        crop_h = np.random.randint(30, min(100, h-y))

                        crop = img[y:y+crop_h, x:x+crop_w]
                        if crop.size > 0:
                            crop_resized = cv2.resize(crop, (IMG_SIZE, IMG_SIZE))
                            cv2.imwrite(os.path.join(non_faces_dir, f'neg_{negative_count}.jpg'), crop_resized)
                            negative_count += 1
                            if negative_count >= 2000:
                                return
            except Exception:
                continue

    # Copy positive samples (faces)
    positive_count = 0
    for person_dir in os.listdir(base_path):
        person_path = os.path.join(base_path, person_dir)
        if os.path.isdir(person_path) and person_dir not in ['faces', 'non_faces', 'face_dataset']:
            for img_file in os.listdir(person_path):
                if img_file.lower().endswith(('.jpg', '.png')):
                    img_path = os.path.join(person_path, img_file)
                    try:
                        img = cv2.imread(img_path)
                        if img is not None:
                            img_resized = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
                            cv2.imwrite(os.path.join(faces_dir, f'face_{positive_count}.jpg'), img_resized)
                            positive_count += 1
                    except Exception:
                        continue

    print(f"Total positive samples (faces): {positive_count}")
    generate_negative_samples()
    negative_count = len(os.listdir(non_faces_dir))
    print(f"Total negative samples (non-faces): {negative_count}")

    # Susun dataset folder akhir
    os.makedirs(os.path.join(final_dataset_dir, 'faces'), exist_ok=True)
    os.makedirs(os.path.join(final_dataset_dir, 'non_faces'), exist_ok=True)

    # Pindahkan semua ke face_dataset agar ImageDataGenerator bisa membacanya
    import shutil
    for f in os.listdir(faces_dir):
        shutil.copy(os.path.join(faces_dir, f), os.path.join(final_dataset_dir, 'faces', f))
    for f in os.listdir(non_faces_dir):
        shutil.copy(os.path.join(non_faces_dir, f), os.path.join(final_dataset_dir, 'non_faces', f))

    return final_dataset_dir


    # Copy positive samples (faces)
    positive_count = 0
    for person_dir in os.listdir(lfw_path):
        person_path = os.path.join(lfw_path, person_dir)
        if os.path.isdir(person_path):
            for img_file in os.listdir(person_path):
                if img_file.endswith(('.jpg', '.png')):
                    img_path = os.path.join(person_path, img_file)
                    try:
                        img = cv2.imread(img_path)
                        if img is not None:
                            img_resized = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
                            cv2.imwrite(f'/home/wahyusetiawan/Documents/ProjectBelajar/facerecognition/DataAnalyst/lfw_funneled/faces/face_{positive_count}.jpg', img_resized)
                            positive_count += 1
                    except Exception as e:
                        continue

    print(f"Total positive samples (faces): {positive_count}")

    # Generate negative samples
    generate_negative_samples()

    negative_count = len(os.listdir('/home/wahyusetiawan/Documents/ProjectBelajar/facerecognition/DataAnalyst/lfw_funneled/' \
    'non_faces'))
    print(f"Total negative samples (non-faces): {negative_count}")

    return '/home/wahyusetiawan/Documents/ProjectBelajar/facerecognition/DataAnalyst/lfw_funneled/face_dataset'

def plot_training_history(history):
    """Plot grafik training history"""
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))

    # Accuracy
    axes[0, 0].plot(history.history['accuracy'], label='Training Accuracy')
    axes[0, 0].plot(history.history['val_accuracy'], label='Validation Accuracy')
    axes[0, 0].set_title('Model Accuracy')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Accuracy')
    axes[0, 0].legend()

    # Loss
    axes[0, 1].plot(history.history['loss'], label='Training Loss')
    axes[0, 1].plot(history.history['val_loss'], label='Validation Loss')
    axes[0, 1].set_title('Model Loss')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Loss')
    axes[0, 1].legend()

    # Precision
    axes[1, 0].plot(history.history['precision'], label='Training Precision')
    axes[1, 0].plot(history.history['val_precision'], label='Validation Precision')
    axes[1, 0].set_title('Model Precision')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Precision')
    axes[1, 0].legend()

    # Recall
    axes[1, 1].plot(history.history['recall'], label='Training Recall')
    axes[1, 1].plot(history.history['val_recall'], label='Validation Recall')
    axes[1, 1].set_title('Model Recall')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('Recall')
    axes[1, 1].legend()

    plt.tight_layout()
    plt.show()

def test_face_detection(model, test_image_path):
    """Test model pada gambar baru"""
    img = cv2.imread(test_image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img_rgb, (IMG_SIZE, IMG_SIZE))
    img_normalized = img_resized / 255.0
    img_batch = np.expand_dims(img_normalized, axis=0)

    prediction = model.predict(img_batch)
    class_names = ['faces', 'non_faces']
    predicted_class = class_names[np.argmax(prediction)]
    confidence = np.max(prediction)

    plt.figure(figsize=(8, 6))
    plt.imshow(img_rgb)
    plt.title(f'Prediction: {predicted_class} (Confidence: {confidence:.2f})')
    plt.axis('off')
    plt.show()

    return predicted_class, confidence

# Main training script
def main():
    print("=== Face Detection Model Training ===")

    # 1. Persiapkan dataset
    print("Mempersiapkan dataset...")
    dataset_path = prepare_lfw_dataset()

    # 2. Buat model
    print("Membuat model CNN...")
    face_detector = FaceDetectionModel(img_size=IMG_SIZE, num_classes=NUM_CLASSES)
    model = face_detector.create_cnn_model()

    # 3. Compile model
    face_detector.compile_model(learning_rate=LEARNING_RATE)

    # 4. Print model summary
    model.summary()

    # 5. Buat data generators
    print("Membuat data generators...")
    train_gen, val_gen = face_detector.create_data_generators(dataset_path)

    # 6. Setup callbacks
    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=5,
            min_lr=1e-7
        ),
        tf.keras.callbacks.ModelCheckpoint(
            '/home/wahyusetiawan/Documents/ProjectBelajar/facerecognition/DataAnalyst/modelai/best_face_detection_model.h5',
            monitor='val_accuracy',
            save_best_only=True,
            verbose=1
        )
    ]

    # 7. Training
    print("Memulai training...")
    print("Train samples:", train_gen.samples)
    print("Validation samples:", val_gen.samples)

    print("Train classes:", train_gen.class_indices)
    print("Validation classes:", val_gen.class_indices)

    history = model.fit(
        train_gen,
        steps_per_epoch=train_gen.samples // BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=val_gen,
        validation_steps=val_gen.samples // BATCH_SIZE,
        callbacks=callbacks,
        verbose=1
    )

    # 8. Plot hasil training
    plot_training_history(history)

    # 9. Save model
    model.save('/home/wahyusetiawan/Documents/ProjectBelajar/DataAnalyst/facerecognition/modelai/face_detection_final_model.h5')
    print("Model disimpan di /home/wahyusetiawan/Documents/facerecognition/ProjectBelajar/DataAnalyst/facerecognition/modelai/face_detection_final_model.h5")

    # 10. Evaluasi model
    print("Evaluasi model pada validation set:")
    val_loss, val_accuracy, val_precision, val_recall = model.evaluate(val_gen)
    print(f"Validation Accuracy: {val_accuracy:.4f}")
    print(f"Validation Precision: {val_precision:.4f}")
    print(f"Validation Recall: {val_recall:.4f}")

    return model, history

# Jalankan training
if __name__ == "__main__":
    # Pastikan GPU tersedia
    print("GPU Available: ", tf.config.list_physical_devices('GPU'))

    # Set memory growth untuk GPU
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
        except RuntimeError as e:
            print(e)

    # Jalankan training
    trained_model, training_history = main()

    print("\n=== Training Selesai ===")
    print("Model siap untuk digunakan!")

    # Contoh penggunaan untuk testing
    # test_face_detection(trained_model, '/path/to/test/image.jpg')



GPU Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
=== Face Detection Model Training ===
Mempersiapkan dataset...
Total positive samples (faces): 13233
Total negative samples (non-faces): 2000
Membuat model CNN...


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_10 (Conv2D)              │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 222, 222, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 109, 109, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 52, 52, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 24, 24, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ (None, 24, 24, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 12, 12, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 10, 10, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_14          │ (None, 10, 10, 512)    │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 5, 5, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 12800)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 12800)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 512)            │     6,554,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 2)              │           514 │
└─────────────────────────────────┴────────────────────────┴─────────────

 Total params: 8,258,498 (31.50 MB)

 Trainable params: 8,256,514 (31.50 MB)

 Non-trainable params: 1,984 (7.75 KB)

Membuat data generators...
Found 12187 images belonging to 2 classes.
Found 3046 images belonging to 2 classes.
Memulai training...
Train samples: 12187
Validation samples: 3046
Train classes: {'faces': 0, 'non_faces': 1}
Validation classes: {'faces': 0, 'non_faces': 1}
Epoch 1/5
55/95 ━━━━━━━━━━━━━━━━━━━━ 1:20 2s/step - accuracy: 0.9314 - loss: 0.5026 - precision: 0.9314 - recall: 0.9314

KeyboardInterrupt: 